In [3]:
import numpy as np 
import cv2
import pandas as pd 
import os 
import pickle

In [4]:
# feature detection
img = cv2.imread('./images/Jennifer Lawrence/007_72ad75ba.jpg')
cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [30]:
# models
net = cv2.dnn.readNetFromCaffe('./models/deploy.prototxt.txt','./models/res10_300x300_ssd_iter_140000_fp16 (2).caffemodel')
descriptor_model = cv2.dnn.readNetFromTorch('./models/openface.nn4.small2.v1.t7')

In [27]:
def helper(image):
    img = cv2.imread(image)
    blog = cv2.dnn.blobFromImage(img,1,(300,300),(104,177,123),swapRB=False,crop=False)
    net.setInput(blog)
    detections = net.forward()
    h,w = img.shape[:2]
    if len(detections > 0 ):
        i = np.argmax(detections[0,0,:,2])
        confidence = detections[0,0,i,2]
        if confidence >0.5:
            box = detections[0,0,i,3:7]*np.array([w,h,w,h])
            startx,starty,endx,endy = box.astype('int')
            roi = img[starty:endy,startx:endx].copy()
            
            #getting face descriptors
            faceblob = cv2.dnn.blobFromImage(roi,1/255,(96,96),(0,255,0),swapRB=True,crop= True)
            descriptor_model.setInput(faceblob)
            vectors = descriptor_model.forward()
            return vectors
    return None         

In [28]:
#create dictionary to store the data and lebel
data = {'data':[],'label':[]}

In [31]:
#Applying helper function to get all images from folder
folders = os.listdir('./images')
for folder in folders:
    path = os.path.join('./images',folder)
    files = os.listdir(path)
    for file in files:
        filepath = os.path.join(path,file)
        try:
            vectors = helper(filepath)
            if vectors is not None:
                data['data'].append(vectors)
                data['label'].append(folder)
                print("feature extracted successfully")
        except Exception as e:
            print(e)        

feature extracted successfully
feature extracted successfully
feature extracted successfully
feature extracted successfully
feature extracted successfully
feature extracted successfully
feature extracted successfully
feature extracted successfully
feature extracted successfully
feature extracted successfully
feature extracted successfully
feature extracted successfully
feature extracted successfully
feature extracted successfully
feature extracted successfully
feature extracted successfully
feature extracted successfully
feature extracted successfully
feature extracted successfully
feature extracted successfully
feature extracted successfully
feature extracted successfully
feature extracted successfully
feature extracted successfully
feature extracted successfully
feature extracted successfully
feature extracted successfully
feature extracted successfully
feature extracted successfully
feature extracted successfully
feature extracted successfully
feature extracted successfully
feature 

In [36]:
pd.Series(data['label']).value_counts()

Scarlett Johansson    200
Tom Hanks             100
Brad Pitt             100
Kate Winslet          100
Jennifer Lawrence     100
Leonardo DiCaprio     100
Megan Fox             100
Nicole Kidman         100
Johnny Depp           100
Robert Downey Jr      100
Tom Cruise             99
Sandra Bullock         99
Denzel Washington      99
Will Smith             99
Natalie Portman        99
Hugh Jackman           98
Angelina Jolie         97
Name: count, dtype: int64

In [37]:
#use picke library to make a pickle file of all extracted feature and lebels to train machine learning model
pickle.dump(data,open('data_face_features.pickle',mode='wb'))